In [1]:
import mysql.connector
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders


In [2]:
# Connect to MySQL server
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="users"
)

In [ ]:
# Email configuration
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'miguelibrahim87@gmail.com'
sender_password = 'Migo9021@!'


In [3]:
# Create cursor to execute queries
mycursor = mydb.cursor()

# Query to get the count of restaurant occurrences and filter by rating
mycursor.execute("SELECT Name, COUNT(*) as count FROM rated_restaurants WHERE RATING > 3 GROUP BY Name HAVING count > 1")

# Fetch all rows of the query result
hotspots = mycursor.fetchall()

# Variable to keep track of the maximum occurrence count
max_count = 0

# Loop through each hotspot to find the maximum count
for hotspot in hotspots:
    # Check if the current hotspot has a higher count than the previous maximum
    if hotspot[1] > max_count:
        max_count = hotspot[1]

# Variable to store the recommended hotspot(s)
recommended_restaurants = []

# Loop through each hotspot to recommend based on maximum count
for hotspot in hotspots:
    # Check if the current hotspot has the maximum count
    if hotspot[1] == max_count:
        recommended_restaurants.append(hotspot[0])

# Print the recommended hotspot(s)
if recommended_restaurants:
    print("Hotspot restaurant(s):")
    for restaurant in recommended_restaurants:
        print(restaurant)

    # Query 'users' table to get email for all users
    mycursor.execute("SELECT email FROM users")
    user_emails = mycursor.fetchall()

    # Create the email message
    subject = "Hotspot Restaurant Recommendation"
    body = f"Dear user,\n\nWe would like to recommend the following hotspot restaurant(s): {', '.join(recommended_restaurants)}.\n\nEnjoy your meal!\n\nBest regards,\nThe Restaurant Recommendation Team"
    message = MIMEMultipart()
    message['From'] = sender_email
    message['Subject'] = subject

    # Attach the body to the email
    message.attach(MIMEText(body, 'plain'))

    # Connect to the SMTP server and send the email to each user
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        for user_email in user_emails:
            message['To'] = user_email[0]
            server.sendmail(sender_email, user_email[0], message.as_string())

    print("Emails sent successfully!")
else:
    print("No hotspot restaurants found.")

No hotspot restaurants found.
